## Calculo de mu combinando Monojet y Dijet

In [5]:
import pyhf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyhf.contrib.viz import brazil

In [6]:
#Lectura de datos
VBF = pd.read_csv("../mu_for_VBF/VBFData.csv")
MJ = pd.read_csv("../mu_for_monojet/MJData.csv")
DJ = pd.read_csv("../mu_for_Dijet/DJData.csv")

In [7]:
bkg = np.concatenate((VBF["VBF_bkg"], DJ["DJ_bkg"], MJ["MJ_bkg"]), axis=None)
errbkg = np.concatenate((VBF["VBF_errbkg"], DJ["DJ_errbkg"], MJ["MJ_errbkg"]), axis=None)
Data = np.concatenate((VBF["VBF_data"], DJ["DJ_data"], MJ["MJ_data"]), axis=None)
sl = np.concatenate((VBF["VBF_ldecays"], DJ["DJ_signal"], MJ["MJ_axial"]), axis=None)
WZ = np.concatenate((VBF["VBF_WZdecays"], DJ["DJ_signal"], MJ["MJ_axial"]), axis=None)

In [8]:
model1 = pyhf.simplemodels.uncorrelated_background(signal = list(sl), bkg = list(bkg), bkg_uncertainty = list(errbkg))
model2 = pyhf.simplemodels.uncorrelated_background(signal = list(WZ), bkg = list(bkg), bkg_uncertainty = list(errbkg))

In [9]:
#Calculamos el mu observado y esperado
poi_values = np.linspace(0.01, 2, 100)

observations1 = np.concatenate((Data, model1.config.auxdata)) #Observaciones modelo 1 - Señal sl (VBF) / Señal axial (MJ)

#Limites - sl
obs_limit1, exp_limits1, (scan1, results1) = pyhf.infer.intervals.upper_limits.upper_limit(
    observations1, model1, poi_values, level=0.05, return_results=True
)

print("Señal sl (VBF) / Señal axial (MJ)")
print(f"Upper limit (obs): μ = {obs_limit1:.4f}")
print(f"Upper limit (exp): μ = {exp_limits1[2]:.4f}")

Señal sl (VBF) / Señal axial (MJ)
Upper limit (obs): μ = 0.2357
Upper limit (exp): μ = 0.2366


In [10]:
poi_values = np.linspace(0.01, 2, 100)
observations2 = np.concatenate((Data, model2.config.auxdata)) #Observaciones modelo 2 - Señal WZ (VBF) / Señal axial (MJ)

#Limites - WZ
obs_limit2, exp_limits2, (scan2, results2) = pyhf.infer.intervals.upper_limits.upper_limit(
    observations2, model2, poi_values, level=0.05, return_results=True
)

print("Señal WZ (VBF) / Señal axial (MJ)")
print(f"Upper limit (obs): μ = {obs_limit2:.4f}")
print(f"Upper limit (exp): μ = {exp_limits2[2]:.4f}")

Señal WZ (VBF) / Señal axial (MJ)
Upper limit (obs): μ = 0.2386
Upper limit (exp): μ = 0.2364


In [11]:
#Calculo del p-value
CLs_obs1, CLs_exp_band1 = pyhf.infer.hypotest(
    obs_limit1, observations1, model1, return_expected_set=True
)

print("Señal sl (VBF) / Señal axial (MJ)")
print(f"  CLs (obs): Cls = {CLs_obs1:.4f}")
print(f"  CLs (exp): CLs (0σ) = {CLs_exp_band1[2]:.4f}")

CLs_obs2, CLs_exp_band2 = pyhf.infer.hypotest(
    obs_limit2, observations2, model2, return_expected_set=True
)

print("Señal WZ (VBF) / Señal axial (MJ)")
print(f"  CLs (obs): Cls = {CLs_obs2:.4f}")
print(f"  CLs (exp): CLs (0σ) = {CLs_exp_band2[2]:.4f}")

Señal sl (VBF) / Señal axial (MJ)
  CLs (obs): Cls = 0.0495
  CLs (exp): CLs (0σ) = 0.0503
Señal WZ (VBF) / Señal axial (MJ)
  CLs (obs): Cls = 0.0493
  CLs (exp): CLs (0σ) = 0.0474
